In [68]:
#Cuántas horas trabajás por semana aproximadamente
#Tu empleador te paga el monotributo

import pandas as pd
import unidecode
import matplotlib.pyplot as plt1
import matplotlib.pyplot as plt2
import numpy as np

categories = {
    "PyMEs y Cooperativas": [
        "Cooperativa", "Empresa Nacional PyME", "Consultora Nacional PyME",
        "Constructora Nacional PyME", "Trabajo para una persona", "Constructora Nacional Chica"
    ],
    "Grandes Empresas Nacionales": [
        "Empresa Nacional Grande", "Constructora Nacional Grande",
        "Comitente Nacional Grande", "Loma Negra", "Empresa de energía", "Empresa de Energía"
    ],
    "Multinacionales": [
        "Empresa Multinacional", "Constructora Extranjera", "Consultora Extranjera",
        "Multinacional", "S.A. proveedor extranjera filial local",
        "Empresa internacional de real estate state",
        "Multinacional proveedora de herramientas y anclajes",
        "Venta de productos para construccion multinacional", "Holding"
    ],
    "Sector Público": [
        "Administración Pública", "Estado", "Contratista de empresa del estado"
    ],
    "Construcción e Infraestructura": [
        "Constructora Nacional Grande", "Constructora Nacional PyME", "Constructora Extranjera",
        "Desarrolladora", "Contratista", "Instalaciónes termomecanicas", "Fábrica grande"
    ],
    "Energía, Minería y Recursos Naturales": [
        "Empresa de energía", "Empresa de Energía", "Minera"
    ],
    "Industria Farmacéutica y Laboratorios": [
        "Farmacéutica nacional", "Laboratorio"
    ],
    "Logística y Transporte": [
        "Aeropuertos", "Puerto", "Ferroviario"
    ],
    "Ambiental": [
        "Ambiental"
    ],
    "Otros/No contesta": [
        np.nan  # np.nan para incluir valores nulos
    ]
}

def standarize_text(text):
    if text == 'Ing Naval':
        text = 'naval'
    return unidecode.unidecode(text).lower().strip()

def calculate_pay_per_hours_ratio(df):
    df = df.copy()
    df['calculated_pay_per_hour'] = round(df['monthly_salary'] / ( df['hours_per_week'] * 4), 2)
    return df

def get_avg_pay_per_hour_by_career(df, contract_type):
    grouped_data_by_career = df.groupby('career')

    # Apply aggregation (e.g., calculate the mean pay per hour for each career)
    #avg_payment = grouped_data_by_career['calculated_pay_per_hour'].mean()

    aggregated = grouped_data_by_career.agg({
        'calculated_pay_per_hour': 'mean',
        'monthly_salary': 'mean',
        'hours_per_week': 'mean',
    })

    aggregated = aggregated.round(2)

    # Convert the aggregated result to a DataFrame (optional)
    avg_payment_df = aggregated.reset_index()

    # Save the result to a CSV file
    #avg_payment_df.to_csv(f"grouped_by_career_{contract_type}.csv", index=False)

def assign_category(employer):
    for category, values in categories.items():
        if employer in values:
            return category
    return "Otros/No contesta"

salarios = pd.read_csv("Salarios_Simplificado.csv", sep=",", engine="python")


#### Cuánto vale la "hora hombre" trabajando en relación de dependencia y trabajando freelance

In [69]:
#Remuneración mensual
salarios = salarios.rename(columns={
    'Marca temporal': 'timestamp',
    'Cuál es tu nivel de estudios': 'study_level',
    'Qué ingeniería estudias/estudiaste': 'career',
    'En qué universidad cursas/cursaste tu carrera de grado': 'university',
    'Género': 'gender',
    'Trabajás en el rubro de la ingeniería que estudias/estudiaste': 'works_in_field',
    'Tenés más de un trabajo': 'more_one_job',
    'Cuál es tu situación particular de contratación': 'contract_type',
    'Es tu primer trabajo en el rubro': 'first_job',
    'Tu empleador te paga el monotributo': 'monotibuto',
    'Tu empleador respeta alguno de los siguientes derechos laborales': 'labor_rights',
    'Cuántas horas trabajás por semana aproximadamente': 'hours_per_week',
    'Cuántos años de experiencia tenés en el rol (o en el rubro pero que sumen para tu rol)': 'experience_years',
    'Tenés otras personas a cargo': 'people_in_charge',
    'Hacés home office': 'home_office',
    'Te dan alguno de estos "beneficios"': 'benefits',
    'Estás enmarcado/a en algún convenio': 'convenio',
    'Te parece que la remuneración que recibís por tu trabajo es acorde a las horas que trabajás y las tareas que realizás': 'fair_pay_hours_tasks',
    'Cómo definís cuánto cobrás tus trabajos': 'how_set_pay',
    'Si podés definirlo: en promedio cuál es tu honorario por hora': 'avg_pay_per_hour',
    'Si querés dejanos algún comentario': 'comments',
    'Edad': 'age',
    '¿tenés otros estudios relacionados con tu carrera': 'career_related_studies',
    'Ubicación de tu trabajo': 'work_location',
    'Empleador': 'employer',
    'Remuneración mensual': 'monthly_salary',
    'Sector': 'sector',
    'Rol o área': 'role_or_area'
})

salarios['career'] = salarios['career'].apply(standarize_text)

#print(salarios['career'].unique().tolist())
#print(salarios['study_level'].unique().tolist())
#print(salarios['contract_type'].unique().tolist())

monotributistas = salarios.loc[salarios['contract_type']=='Monotributista']
relacion_dependencia = salarios.loc[salarios['contract_type']=='Relación de dependencia registrada']
informal = salarios.loc[salarios['contract_type']=='Informal']
#13 + 564 + 92

monotributistas = calculate_pay_per_hours_ratio(monotributistas)
relacion_dependencia = calculate_pay_per_hours_ratio(relacion_dependencia)
informal = calculate_pay_per_hours_ratio(informal)
#print(monotributistas['calculated_pay_per_hour'])

get_avg_pay_per_hour_by_career(monotributistas, 'monotributista')
get_avg_pay_per_hour_by_career(informal, 'informal')
get_avg_pay_per_hour_by_career(relacion_dependencia, 'relacion_dependencia')


#### Cuánto cobran por rol, por sector de trabajo. Diseccionar por género

In [70]:
print((salarios['gender'].unique().tolist()))

salarios["employer_category"] = salarios["employer"].apply(assign_category)

salaries_per_employer_category = (
    salarios.groupby("employer_category")
    .agg(
        avg_salary=("monthly_salary", lambda x: round(x.mean(), 2)),
        avg_womens_salary=("monthly_salary", lambda x: round(x[salarios["gender"] == "Mujer"].mean(), 2)),
        avg_mens_salary=("monthly_salary", lambda x: round(x[salarios["gender"] == "Hombre"].mean(), 2)),
        avg_nonb_salary=("monthly_salary", lambda x: round(x[salarios["gender"] == 'X'].mean(), 2)),
        count=("monthly_salary", "size")
    )
    .reset_index()
)

salaries_per_employer_category.to_csv(f"salaries_per_employer.csv", index=False)

['Mujer', 'Hombre', 'X']
